In [1]:
!pip install --upgrade --quiet datasets[audio]
!pip install --upgrade --quiet transformers
!pip install --upgrade --quiet accelerate
!pip install --upgrade --quiet bitsandbytes
!pip install --upgrade --quiet peft
!pip install --upgrade --quiet evaluate
!pip install --upgrade --quiet jiwer
!pip install --upgrade --quiet wandb
!pip install --upgrade --quiet tensorboard
!pip install --upgrade --quiet gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 M

In [2]:
import os


os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
import warnings

warnings.filterwarnings("ignore")

In [4]:
from huggingface_hub.hf_api import HfFolder
import os
from google.colab import userdata
import getpass

# os.environ["HF_TOKEN"] = getpass.getpass("Enter Gemini API Key: ")

os.environ["HF_TOKEN"] = userdata.get("HUGGINGFACE_TOKEN")
HfFolder.save_token(os.environ["HF_TOKEN"])

In [5]:
import os
import wandb


os.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")
wandb.login(key=os.environ["WANDB_API_KEY"])

wandb_project = "Whisper-large-V3-Arabic"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mohamed-ahmed. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [7]:
import gc

def clear_gpu():
    torch.clear_autocast_cache()
    torch.cuda.ipc_collect()
    torch.cuda.empty_cache()
    gc.collect()

if device == "gpu":
    clear_gpu()

In [8]:
import multiprocessing

def optimal_workers():
    num_cpus = multiprocessing.cpu_count()
    num_gpus = torch.cuda.device_count()
    optimal_value = min(num_cpus, num_gpus*4) if num_gpus else num_cpus - 1
    return optimal_value

num_cpus = optimal_workers()
num_cpus

2

In [9]:
from datasets import load_dataset

eg_dataset = load_dataset("MightyStudent/Egyptian-ASR-MGB-3")

README.md:   0%|          | 0.00/3.27k [00:00<?, ?B/s]

(…)-00000-of-00002-69f2a5d14e629dff.parquet:   0%|          | 0.00/467M [00:00<?, ?B/s]

(…)-00001-of-00002-860b0ca7a1fb4dc3.parquet:   0%|          | 0.00/454M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1159 [00:00<?, ? examples/s]

In [10]:
eg_dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 1159
    })
})

In [11]:
eg_dataset["train"][0]["sentence"]

'عزيزي المشاهد البرنامج ده مش برنامج كوميدي وبس ده حالة انسانية استضفنا في البرنامج ده ناس كتير منهم الفنان الصحفي سواق التاكسي و المحامي وغيرهم كتير اضطرينا نعرضهم لضغوط كتير زي ما بيحصل في الواقع بالظبط عشان نعرف قد إيه هما بيحبوا البلد دي تعالوا نشوف النهارده هيحصل إيه'

In [12]:
eg_dataset["train"][0]["audio"]

{'path': 'comedy_78_first_12min_part_0.wav',
 'array': array([ 0.        ,  0.        ,  0.        , ..., -0.26385498,
        -0.27554321, -0.29098511]),
 'sampling_rate': 16000}

In [13]:
# eg_dataset = eg_dataset["train"].select(range(100))

In [14]:
eg_dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 1159
    })
})

In [15]:
eg_dataset = eg_dataset["train"].train_test_split(test_size=0.1)
eg_dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 116
    })
})

In [16]:
model_id = "openai/whisper-large-v3"
language="Arabic"
task="transcribe"

In [17]:
from transformers import WhisperFeatureExtractor


feature_extractor = WhisperFeatureExtractor.from_pretrained(model_id)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [18]:
from transformers import WhisperTokenizer


tokenizer = WhisperTokenizer.from_pretrained(model_id,
                                             language=language,
                                             task=task)

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

In [19]:
from transformers import WhisperProcessor


processor = WhisperProcessor.from_pretrained(model_id,
                                             language=language,
                                             task=task)

In [20]:
# from datasets import Audio

# eg_dataset = eg_dataset.cast_column("audio", Audio(sampling_rate=16000))

In [21]:
def prepare_dataset(batch):
    # compute log-Mel input features from input audio array
    input_features = [feature_extractor(sample["array"],
                                       sampling_rate=sample["sampling_rate"]).input_features[0] for sample in batch["audio"]]
    batch["input_features"] = input_features

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [22]:
train_dataset = eg_dataset["train"]
test_dataset = eg_dataset["test"]

In [23]:
del eg_dataset

In [24]:
################################ JUST SMALL SET ###############################
train_dataset = train_dataset.select(range(100))
test_dataset = test_dataset.select(range(2))

In [25]:
train_dataset = train_dataset.map(function=prepare_dataset,
                                  batched=True,
                                  batch_size=128,
                                  num_proc=num_cpus)

test_dataset = test_dataset.map(function=prepare_dataset,
                                batched=True,
                                batch_size=128,
                                num_proc=num_cpus)

Map (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

In [26]:
with_quantization_config = True
with_lora_peft = True

In [27]:
from transformers import WhisperForConditionalGeneration
from transformers import BitsAndBytesConfig


bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.float16)

attn_implementation = "eager"
# attn_implementation="flash_attention_2"

model = WhisperForConditionalGeneration.from_pretrained(model_id,
                                                        quantization_config= bnb_config if with_quantization_config else None,
                                                        attn_implementation=attn_implementation,
                                                        device_map="auto")

model.generation_config.pad_token_id = tokenizer.pad_token_id

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

In [28]:
import bitsandbytes as bnb


def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
        if "lm_head" in lora_module_names: # Needed for 16bit
            lora_module_names.remove("lm_head")

    return list(lora_module_names)

In [29]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


if with_lora_peft:
    target_modules = find_all_linear_names(model)

    qlora_config = LoraConfig(r=128,  # dimension of the updated matrices
                            lora_alpha=32,  # parameter for scaling
                            target_modules=target_modules,  # this chooses on which layers QLoRA is applied
                            lora_dropout=0.05,  # dropout probability for layers
                            bias="none")
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, qlora_config)
    model.print_trainable_parameters()



trainable params: 230,686,720 || all params: 1,774,177,280 || trainable%: 13.0025


In [30]:
model.generation_config.language = language
model.generation_config.task = task
model.generation_config.forced_decoder_ids = None

In [31]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [32]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor,
                                                     decoder_start_token_id=model.config.decoder_start_token_id)

In [33]:
import evaluate

metric = evaluate.load("wer")

In [34]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [35]:
from transformers import Seq2SeqTrainingArguments


output_dir = "Whisper-large-V3-Arabic"

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=5,
    num_train_epochs=1,
    # max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    bf16=False,
    eval_strategy="steps",
    predict_with_generate=True,
    generation_max_length=1024,
    eval_accumulation_steps=1,
    save_steps=10,
    eval_steps=10,
    logging_steps=10,
    report_to=["wandb", "tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,

)

In [36]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor
)

In [37]:
processor.save_pretrained(training_args.output_dir)

[]

In [38]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
10,0.685200,0.706164,45.714286
20,0.750400,0.699449,45.714286
30,0.721900,0.694049,45.714286
40,0.697100,0.690454,45.714286
50,0.684600,0.688955,38.095238


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


TrainOutput(global_step=50, training_loss=0.7078168392181396, metrics={'train_runtime': 832.1711, 'train_samples_per_second': 0.12, 'train_steps_per_second': 0.06, 'total_flos': 3.92900050944e+17, 'train_loss': 0.7078168392181396, 'epoch': 1.0})

In [39]:
kwargs = {
    "dataset_tags": "MightyStudent/Egyptian-ASR-MGB-3",
    "language": "ar",
    "model_name": "Whisper large V3 Arabic",  # a 'pretty' name for our model
    "finetuned_from": model_id,
    "tasks": "automatic-speech-recognition",
}

In [40]:
trainer.push_to_hub(**kwargs)

CommitInfo(commit_url='https://huggingface.co/MohamedAhmedAE/Whisper-large-V3-Arabic/commit/50e1f9aba2025fdca493cac6dc94f683c79b453d', commit_message='End of training', commit_description='', oid='50e1f9aba2025fdca493cac6dc94f683c79b453d', pr_url=None, pr_revision=None, pr_num=None)

In [41]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="MohamedAhmedAE/Whisper-large-V3-Arabic")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(sources=["microphone"], type="filepath"),
    outputs="text",
    title="Whisper large V3 Arabic",
    description="Realtime demo for arabic speech recognition using a fine-tuned Whisper large V3 model.",
)

iface.launch()

adapter_config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://aaa5af3f2bf2e7dd7a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
